In [1]:
'''
MAIN PROMPT: Investigating if the age of a person has a higher chance of 
winning an Oscar or not for each category
'''
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as urllib2

df = pd.read_csv("oscarnominees.csv", encoding="latin1")
df


,Title,Winner,Award,Year
0,Nomadland,Y,Picture,2020
1,Judas and the Black Messiah,N,Picture,2020
2,Mank,N,Picture,2020
3,Minari,N,Picture,2020
4,Promising Young Woman,N,Picture,2020
...,...,...,...,...
2339,Janet Gaynor,Y,Actress,1928
2340,Louise Dresser,N,Actress,1928
2341,Janet Gaynor,N,Actress,1928
2342,Janet Gaynor,N,Actress,1928


In [2]:
# Drop Picture Category
df = df.drop(df[df["Award"] == "Picture"].index)
df

,Title,Winner,Award,Year
8,Anthony Hopkins,Y,Actor,2020
9,Riz Ahmed,N,Actor,2020
10,Chadwick Boseman,N,Actor,2020
11,Gary Oldman,N,Actor,2020
12,Steven Yeun,N,Actor,2020
...,...,...,...,...
2339,Janet Gaynor,Y,Actress,1928
2340,Louise Dresser,N,Actress,1928
2341,Janet Gaynor,N,Actress,1928
2342,Janet Gaynor,N,Actress,1928


In [3]:
print(set(df["Award"]))

{'Actress', 'Actor', 'Supporting Actress', 'Supporting Actor'}


In [4]:
# Wikipedia links
# https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor
# https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress
# https://en.wikipedia.org/wiki/Academy_Award_for_Best_Supporting_Actor
# https://en.wikipedia.org/wiki/Academy_Award_for_Best_Supporting_Actress

# FAILS:
# For best actor, nick nolte failed due to different HTML layout for age
# For best actress, rosamund pike, grace moore failed due to different HTML layout for age (need confirmation)
# Marie-Christine Barrault has a weird case where it didn't go through? it would be /wiki/Marie-Christine_Barrault
# joan crawford has an uncertain year of birth on wikipedia
# ralph richardson, John Gielgud, Stanley Holloway has different HTML
# katina paxinou failed due to different HTML
# every supporting actress besides katina had no DOB
import chardet

# Loads the page of a URL and downloads it HTML contents to be used
def loadPage(url):
    page = urllib2.urlopen(url)
    data = BeautifulSoup(page, "html.parser")

    return data

In [74]:
# Search for a person given a URL and find info about their DOB
def searchActor(url):
    try:
        data = loadPage(url)

        birth_date = data.find("span", {"class": "bday"})

        if birth_date is None:
            birth_date = data.find_all("p")[1]
        
        if "Nick_Nolte" in url:
            birth_year = int(birth_date.text[38:42])

        else:
            birth_year = int(birth_date.text[:4])

    except ValueError as e:
        return "None"

    return birth_year

actorData = loadPage("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor")
'''
def findActor(name):
    name = name.encode("iso-8859-1").decode("utf-8")

    # Find the "Winners and nominees" table
    tables = actorData.find_all("table", {"class": "wikitable"})

    for table in tables:
        rows = table.find_all("tr")

        for row in rows:
            columns = row.find_all("td")

            for column in columns:
                aElement = column.find("a", href=True)
                
                if aElement is not None and name in aElement:
                    href = aElement.get("href")
                    
                    personLink = "https://en.wikipedia.org" + href
                    birthYear = searchActor(personLink)

                    print(name, ":", birthYear)
                    return birthYear

    print(name, ":", "Fail!")
    return 1
'''

actorAge = {}
def findActor(name):
    name = name.encode("iso-8859-1").decode("utf-8")
    if name in actorAge:
        return actorAge[name]
    
    tables = actorData.find_all("a", {"title": name }, href = f'/wiki/{name.replace(" ", "_")}')

    for table in tables:
        if table is not None:
            href = table.get("href")

            personLink = "https://en.wikipedia.org" + href
            birthYear = searchActor(personLink)

            actorAge[name] = birthYear

            print(name, ":", birthYear)

            return birthYear


actors = df[df["Award"] == "Actor"]
actors["Age"] = actors["Title"].apply(findActor)

#actors = df[df["Award"] == "Actor"]
#actors["Age"] = actors["Title"].apply(findActor)

Anthony Hopkins : 1937
Riz Ahmed : 1982
Chadwick Boseman : 1976
Gary Oldman : 1958
Steven Yeun : 1983
Joaquin Phoenix : 1974
Leonardo DiCaprio : 1974
Adam Driver : 1983
Antonio Banderas : 1960
Jonathan Pryce : 1947
Rami Malek : 1981
Christian Bale : 1974
Bradley Cooper : 1975
Willem Dafoe : 1955
Viggo Mortensen : 1958
Daniel Day-Lewis : 1957
Daniel Kaluuya : 1989
Denzel Washington : 1954
Casey Affleck : 1975
Andrew Garfield : 1983
Ryan Gosling : 1980
Bryan Cranston : 1956
Matt Damon : 1970
Michael Fassbender : 1977
Eddie Redmayne : 1982
Benedict Cumberbatch : 1976
Michael Keaton : 1951
Steve Carell : 1962
Matthew McConaughey : 1969
Bruce Dern : 1936
Chiwetel Ejiofor : 1977
Hugh Jackman : 1968
Jean Dujardin : 1972
George Clooney : 1961
Brad Pitt : 1963
Colin Firth : 1960
Javier Bardem : 1969
Jeff Bridges : 1949
Jesse Eisenberg : 1983
James Franco : 1978
Morgan Freeman : 1937
Jeremy Renner : 1971
Sean Penn : 1960
Richard Jenkins : 1947
Frank Langella : 1938
Mickey Rourke : 1952
Johnny De

C:\Users\tommy\AppData\Local\Temp\ipykernel_9172\3323672315.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actors["Age"] = actors["Title"].apply(test)


In [78]:
actressData = loadPage("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress")

def searchActress(url):
    try:
        data = loadPage(url)

        birth_date = data.find("span", {"class": "bday"})

        if birth_date is None:
            birth_date = data.find_all("div", {"class": "shortdescription nomobile noexcerpt noprint searchaux"})[0]

        if "Rosamund_Pike" in url:
            birth_year = int(birth_date.text[-5:-1])
        elif "Grace_Moore" in url or "Joan_Crawford" in url:
            birth_year = int(birth_date.text[-10:-6])

        else:
            birth_year = int(birth_date.text[:4])
        
        return birth_year
    except ValueError as e:
        return "None"

    
'''
def findActress(name):
    name = name.encode("iso-8859-1").decode("utf-8")

    # Find the "Winners and nominees" table
    tables = actressData.find_all("table", {"class": "wikitable"})

    for table in tables:
        rows = table.find_all("tr")

        for row in rows:
            columns = row.find_all("td")

            for column in columns:
                aElement = column.find("a", href=True)

                if aElement is not None and name in aElement.text:
                    href = aElement.get("href")
                    
                    personLink = "https://en.wikipedia.org" + href
                    birthYear = searchActress(personLink, name)

                    print(name, ":", birthYear)
                    return birthYear

    print(name, ":", "Fail!")
    return 1
'''

actressAge = {}
def findActress(name):
    name = name.encode("iso-8859-1").decode("utf-8")
    if name in actressAge:
        return actressAge[name]
    
    tables = actressData.find_all("a", {"title": name }, href = f'/wiki/{name.replace(" ", "_")}')

    for table in tables:
        if table is not None:
            href = table.get("href")

            personLink = "https://en.wikipedia.org" + href
            birthYear = searchActress(personLink)

            actressAge[name] = birthYear

            print(name, ":", birthYear)

            return birthYear

actress = df[df["Award"] == "Actress"]
actress["Age"] = actress["Title"].apply(findActress)


Frances McDormand : 1957
Viola Davis : 1965
Andra Day : 1984
Vanessa Kirby : 1988
Carey Mulligan : 1985
Cynthia Erivo : 1987
Scarlett Johansson : 1984
Saoirse Ronan : 1994
Charlize Theron : 1975
Olivia Colman : 1974
Yalitza Aparicio : 1993
Glenn Close : 1947
Lady Gaga : 1986
Melissa McCarthy : 1970
Sally Hawkins : 1976
Margot Robbie : 1990
Meryl Streep : 1949
Emma Stone : 1988
Isabelle Huppert : 1953
Ruth Negga : 1981
Natalie Portman : 1981
Brie Larson : 1989
Cate Blanchett : 1969
Jennifer Lawrence : 1990
Charlotte Rampling : 1946
Julianne Moore : 1960
Felicity Jones : 1983
Marion Cotillard : 1975
Reese Witherspoon : 1976
Rosamund Pike : 1979
Amy Adams : 1974
Sandra Bullock : 1964
Judi Dench : 1934
Jessica Chastain : 1977
Emmanuelle Riva : 1927
Naomi Watts : 1968
Rooney Mara : 1985
Annette Bening : 1958
Nicole Kidman : 1967
Helen Mirren : 1945
Gabourey Sidibe : 1983
Kate Winslet : 1975
Anne Hathaway : 1982
Angelina Jolie : 1975
Melissa Leo : 1960
Julie Christie : 1940
Laura Linney : 19

C:\Users\tommy\AppData\Local\Temp\ipykernel_9172\4293940568.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actress["Age"] = actress["Title"].apply(test)


In [21]:
actress = actress.drop(actress[actress["Age"] == "None"].index)

actress

,Title,Winner,Award,Year,Age
13,Frances McDormand,Y,Actress,2020,1957
14,Viola Davis,N,Actress,2020,1965
15,Andra Day,N,Actress,2020,1984
16,Vanessa Kirby,N,Actress,2020,1988
17,Carey Mulligan,N,Actress,2020,1985
...,...,...,...,...,...
2339,Janet Gaynor,Y,Actress,1928,1906
2340,Louise Dresser,N,Actress,1928,1878
2341,Janet Gaynor,N,Actress,1928,1906
2342,Janet Gaynor,N,Actress,1928,1906


In [81]:
bestSupportingActorData = loadPage("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Supporting_Actor")

def searchSupportingActor(url):
    try:
        data = loadPage(url)

        birth_date = data.find("span", {"class": "bday"})

        if birth_date is None:
            birth_date = data.find_all("p")

        if "Ralph_Richardson" in url:
            birth_year = int(birth_date[2].text[40:44])

        elif "John_Gielgud" in url:
            birth_year = int(birth_date[5].text[29:33])

        elif "Stanley_Holloway" in url:
            birth_year = int(birth_date[1].text[41:45])
            
        elif "Nick_Nolte" in url:
            birth_date = data.find_all("p")[1].text
            birth_year = birth_date[38:42]
            
        else:
            birth_year = int(birth_date.text[:4])
        
        return birth_year
    
    except ValueError as e:
        return "None"

def findSupportingActor(name):
    name = name.encode("iso-8859-1").decode("utf-8")

    # Find the "Winners and nominees" table
    tables = bestSupportingActorData.find_all("table", {"class": "wikitable"})

    for table in tables:
        rows = table.find_all("tr")

        for row in rows:
            columns = row.find_all("td")

            for column in columns:
                aElement = column.find("a", href=True)
                
                if aElement is not None and name in aElement:
                    href = aElement.get("href")
                    
                    personLink = "https://en.wikipedia.org" + href
                    birthYear = searchSupportingActor(personLink, name)

                    print(name, ":", birthYear)
                    return birthYear

    print(name, ":", "Fail!")
    return 1

supportingActorAge = {}
def test(name):
    name = name.encode("iso-8859-1").decode("utf-8")
    if name in supportingActorAge:
        return supportingActorAge[name]
    
    tables = bestSupportingActorData.find_all("a", {"title": name }, href = f'/wiki/{name.replace(" ", "_")}')

    for table in tables:
        if table is not None:
            href = table.get("href")

            personLink = "https://en.wikipedia.org" + href
            birthYear = searchSupportingActor(personLink)

            supportingActorAge[name] = birthYear

            print(name, ":", birthYear)

            return birthYear

bestSupportingActor = df[df["Award"] == "Supporting Actor"]

bestSupportingActor["Age"] = bestSupportingActor["Title"].apply(test)

Daniel Kaluuya : 1989
Sacha Baron Cohen : 1971
Leslie Odom Jr. : 1981
Paul Raci : 1948
LaKeith Stanfield : 1991
Brad Pitt : 1963
Tom Hanks : 1956
Anthony Hopkins : 1937
Al Pacino : 1940
Joe Pesci : 1943
Mahershala Ali : 1974
Adam Driver : 1983
Sam Elliott : 1944
Richard E. Grant : 1957
Sam Rockwell : 1968
Willem Dafoe : 1955
Woody Harrelson : 1961
Richard Jenkins : 1947
Christopher Plummer : 1929
Jeff Bridges : 1949
Lucas Hedges : 1996
Dev Patel : 1990
Michael Shannon : 1974
Mark Rylance : 1960
Christian Bale : 1974
Tom Hardy : 1977
Mark Ruffalo : 1967
Sylvester Stallone : 1946
J. K. Simmons : 1955
Edward Norton : 1969
Ethan Hawke : 1970
Robert Duvall : 1931
Jared Leto : 1971
Barkhad Abdi : 1985
Bradley Cooper : 1975
Michael Fassbender : 1977
Jonah Hill : 1983
Alan Arkin : 1934
Robert De Niro : 1943
Philip Seymour Hoffman : 1967
Tommy Lee Jones : 1946
Christoph Waltz : 1956
Kenneth Branagh : 1960
Nick Nolte : 1941
Max von Sydow : 1929
Jeremy Renner : 1971
Geoffrey Rush : 1951
Matt Damo

C:\Users\tommy\AppData\Local\Temp\ipykernel_9172\1773657839.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bestSupportingActor["Age"] = bestSupportingActor["Title"].apply(test)


In [82]:
bestSupportingActressData = loadPage("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Supporting_Actress")

def searchSupportingActress(url):
    try:
        data = loadPage(url)
        birth_date = data.find("span", {"class": "bday"})

        if birth_date is None:
            birth_date = data.find_all("p")

        if "Rachel_Griffiths" in url:
            birth_year = int(birth_date[1].text[34:38])
        
        elif "Katina_Paxinou" in url:
            birth_year = int(birth_date[0].text[51:55])

        else:
            birth_year = int(birth_date.text[:4])
        
        return birth_year
    except AttributeError as e:
        return "None"
    
'''
def findSupportingActress(name):
    name = name.encode("iso-8859-1").decode("utf-8")

    # Find the "Winners and nominees" table
    tables = bestSupportingActressData.find_all("table", {"class": "wikitable"})

    for table in tables:
        rows = table.find_all("tr")

        for row in rows:
            columns = row.find_all("td")

            for column in columns:
                aElement = column.find("a", href=True)
                
                if aElement is not None and name in aElement:
                    href = aElement.get("href")

                    personLink = "https://en.wikipedia.org" + href
                    birthYear = searchSupportingActress(personLink)

                    print(name, ":", birthYear)
                    return birthYear

    print(name, ":", "Fail!")
    return 1
'''

supportingActressAge = {}
def findSupportingActress(name):
    name = name.encode("iso-8859-1").decode("utf-8")
    if name in supportingActressAge:
        return supportingActressAge[name]
    
    tables = bestSupportingActressData.find_all("a", {"title": name }, href = f'/wiki/{name.replace(" ", "_")}')

    for table in tables:
        if table is not None:
            href = table.get("href")

            personLink = "https://en.wikipedia.org" + href
            birthYear = searchSupportingActress(personLink)

            supportingActressAge[name] = birthYear

            print(name, ":", birthYear)

            return birthYear
        
bestSupportingActress = df[df["Award"] == "Supporting Actress"]
bestSupportingActress["Age"] = bestSupportingActress["Title"].apply(findSupportingActress)

Youn Yuh-jung : 1947
Maria Bakalova : 1996
Glenn Close : 1947
Olivia Colman : 1974
Amanda Seyfried : 1985
Laura Dern : 1967
Kathy Bates : 1948
Scarlett Johansson : 1984
Florence Pugh : 1996
Margot Robbie : 1990
Regina King : 1971
Amy Adams : 1974
Marina de Tavira : 1974
Emma Stone : 1988
Rachel Weisz : 1970
Allison Janney : 1959
Mary J. Blige : 1971
Lesley Manville : 1956
Laurie Metcalf : 1955
Octavia Spencer : 1970
Viola Davis : 1965
Naomie Harris : 1976
Nicole Kidman : 1967
Alicia Vikander : 1988
Jennifer Jason Leigh : 1962
Rooney Mara : 1985
Rachel McAdams : 1978
Kate Winslet : 1975
Patricia Arquette : 1968
Keira Knightley : 1985
Meryl Streep : 1949
Sally Hawkins : 1976
Jennifer Lawrence : 1990
Julia Roberts : 1967
June Squibb : 1929
Sally Field : 1946
Anne Hathaway : 1982
Helen Hunt : 1963
Jacki Weaver : 1947
Jessica Chastain : 1977
Melissa McCarthy : 1970
Janet McTeer : 1961
Melissa Leo : 1960
Helena Bonham Carter : 1966
Hailee Steinfeld : 1996
Vera Farmiga : 1973
Maggie Gyllenhaa

C:\Users\tommy\AppData\Local\Temp\ipykernel_9172\943976792.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bestSupportingActress["Age"] = bestSupportingActress["Title"].apply(test)


In [45]:

bestSupportingActress = bestSupportingActress.drop(bestSupportingActress[bestSupportingActress["Age"] == "None"].index)
bestSupportingActress

,Title,Winner,Award,Year,Age
23,Youn Yuh-jung,Y,Supporting Actress,2020,1947
24,Maria Bakalova,N,Supporting Actress,2020,1996
25,Glenn Close,N,Supporting Actress,2020,1947
26,Olivia Colman,N,Supporting Actress,2020,1974
27,Amanda Seyfried,N,Supporting Actress,2020,1985
52,Laura Dern,Y,Supporting Actress,2019,1967
53,Kathy Bates,N,Supporting Actress,2019,1948
54,Scarlett Johansson,N,Supporting Actress,2019,1984
55,Florence Pugh,N,Supporting Actress,2019,1996
56,Margot Robbie,N,Supporting Actress,2019,1990
